# Atlas Reader

[Atlas Reader](https://github.com/miykael/atlasreader) is a Python interface for generating coordinate tables and region labels from statistical MRI images.

## Installing `atlasreader`

Provided you have `pip` at your disposal, installing `atlasreader` is as simple as this:

```pip install -U atlasreader```


## Using `atlasreader`

`atlasreader` can either be run through the command line interface, or directly within Python. To showcase both of those examples, let's first import relevant plotting functions for this notebook.

In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
from nilearn import plotting
from IPython.display import Image

Next, let's download some example data via `nilearn`.

In [ ]:
from nilearn.datasets import fetch_neurovault_motor_task

In [ ]:
# get a stat map from neurovault using nilearn
motor_images = fetch_neurovault_motor_task()
stat_img = motor_images.images[0]

And what does our example data look like?

In [ ]:
plotting.plot_glass_brain(stat_img, black_bg=True, threshold=5,
                          vmax=10, colorbar=True, plot_abs=False,
                          title='Finger tapping task')

Ok, so we can see that there are a few particular clusters, if we threshold the data at a value of 5.

`atlasreader` gives you now the opportunity to better understand where the peaks of those clusters are, over which regions the clusters extend, and much more. So, how is it done?

## Calling `atlasreader` from Python

If you want run `atlasreader` directly in Python, just import the `create_output()` function, and you're good to go:

In [ ]:
from atlasreader.atlasreader import create_output

Let's for example say that you want to get the atlas information of the statistical map from above, thresholded at a value of 5, but only for the [AAL](http://neuro.imm.dtu.dk/wiki/Automated_Anatomical_Labeling) atlas.

In [ ]:
create_output(stat_img,
              atlas='aal',
              voxel_thresh=5,
              cluster_extent=0,
              prob_thresh=0,
              outdir='.')

As an output we get four different kind of files.

*First*, an overview glass brain plot, with the name of `stat_img` but with the file ending `.png`

In [ ]:
Image('image_10426.png')

*Second*, a statistical map plot for each cluster found in the data, centered around the peak of this cluster. Those files all end with `_cluster01.png`, `_cluster02.png`, ...:

In [ ]:
Image('image_10426_cluster01.png')

In [ ]:
Image('image_10426_cluster02.png')

*Third*, a csv file ending with `_peaks.csv`, containing the location of each cluster's peak, it's signal value at this location and the atlas correspondence:

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv('image_10426_peaks.csv')

*Forth*, a csv file ending with `_clusters.csv`, containing the location of each cluster's peak, it's mean value, its cluster extend, as well as the membership of each cluster, given a particular atlas.

In [ ]:
pd.read_csv('image_10426_clusters.csv')

For example, in the csv table shown above, we know that 51.68% of the first cluster is in the righ postcentral and 28.75% is in the right precentral area, according to the AAL atlas.

### Adapting cluster parameters

As you can see, there are some clusters with a cluster extend of less than 5000 mm³ (i.e. with a voxel resolution of 3mm x 3mm x 3mm this corresponds to about 185 voxels.

So, let's say we want to ignore those clusters with less than 185 voxles / 5000 mm³ and get the atlas information for FreeSurfer's *Desikan_Killiany* and FSL's *Harvard-Oxford* atlas. To do this, we can adapt the parameters as follows:

In [ ]:
create_output(stat_img,
              atlas=['Desikan_Killiany', 'Harvard_Oxford'],
              voxel_thresh=5,
              cluster_extent=185,
              prob_thresh=0,
              outdir='.')

Plotting the glass brain again, we see that only the biggest clusters have survived the restrictions:

In [ ]:
Image('image_10426.png')

And looking at the two CSV tables, we can see that we now have peak and cluster information for two atlases:

In [ ]:
pd.read_csv('image_10426_peaks.csv')

In [ ]:
pd.read_csv('image_10426_clusters.csv')

## `atlasreader` from the Command Line

This is all super exciting! But as promised before, `atlasreader` can also be run directly from the command line. Assuming you installed it via `pip`.

So let's see what the options are:

In [ ]:
%%bash 
atlasreader -h 

Perfect! As you can see, we have the same input parameters available as if we would run `atlasreader` within Python.

So let's try something new out. Let's set the *probability threshold* to 40% and the *atlas* to `all`. The rest we can keep as in the example before:

In [ ]:
%%bash -s "$stat_img"
atlasreader -a all -t 5 -c 185 -p 40 -o . $1

The glassbrain will look the same, but what about the csv tables?

In [ ]:
pd.read_csv('image_10426_peaks.csv')

As we can see, we now get atlas information for all the atlases included in `atlasreader`. Currently those are AAL, FreeSurfer's Desikan-Killiany& Destrieux, FSL's Harvard-Oxford, Juelich and Neuromorphometrics atlas. This is due to the fact that we set the parameter `atlas` to `'all`.

But what about the `probability threshold` parameter at 40%? This means that only atlas memberships of higher than 40% are shown in the tables:

In [ ]:
pd.read_csv('image_10426_clusters.csv')

That this is the case, becomes more eminent, if we run the same command again, but this time with the `probability threshold` parameters set to zero.

In [ ]:
%%bash -s "$stat_img"
atlasreader -a all -t 5 -c 185 -p 0 -o . $1

In [ ]:
pd.read_csv('image_10426_clusters.csv')